In [2]:
pip install pandas sqlalchemy ipython-sql prettytable


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade ipython-sql


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade prettytable

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Obtaining dependency information for prettytable from https://files.pythonhosted.org/packages/73/19/4bb9530512432774fdd7cb7c020851d4decbb811d95f86fd4f6a870a6d3e/prettytable-3.12.0-py3-none-any.whl.metadata
  Using cached prettytable-3.12.0-py3-none-any.whl.metadata (30 kB)
Using cached prettytable-3.12.0-py3-none-any.whl (31 kB)
  Attempting uninstall: prettytable
    Found existing installation: prettytable 2.5.0
    Uninstalling prettytable-2.5.0:
      Successfully uninstalled prettytable-2.5.0


In [5]:
# Import required libraries for database connection and data manipulation
import sqlite3
import pandas as pd


In [15]:
import prettytable
prettytable.__dict__['DEFAULT'] = prettytable.PLAIN_COLUMNS


C:\Users\Zaki Bouaoudia\AppData\Local\Temp\ipykernel_58772\1122509105.py:2: DeprecationWarning: the 'PLAIN_COLUMNS' constant is deprecated, use the 'TableStyle' enum instead
  prettytable.__dict__['DEFAULT'] = prettytable.PLAIN_COLUMNS


In [6]:
# List of CSV file paths
csv_files = {
    "Appearances": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\appearances.csv",
    "Games": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\games.csv",
    "Leagues": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\leagues.csv",
    "Players": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\players.csv",
    "Shots": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\shots.csv",
    "Teams": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\teams.csv",
    "TeamStats": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\teamstats.csv",
}

In [7]:
# Create SQLite database (file-based)
db_file = "football_database.db"
conn = sqlite3.connect(db_file)  # Use a persistent database file


In [8]:
# Load each CSV into a separate table
for table_name, file_path in csv_files.items():
    try:
        # Specify a fallback encoding
        df = pd.read_csv(file_path, encoding='latin1')
        df.to_sql(table_name, conn, index=False, if_exists='replace')
        print(f"Loaded {file_path} into table {table_name}")
    except UnicodeDecodeError as e:
        print(f"Error loading {file_path}: {e}")


Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\appearances.csv into table Appearances
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\games.csv into table Games
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\leagues.csv into table Leagues
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\players.csv into table Players
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\shots.csv into table Shots
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\teams.csv into table Teams
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\teamstats.csv into table TeamStats


In [9]:
# Verify tables are loaded
print("Tables in the database:")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

Tables in the database:
          name
0  Appearances
1        Games
2      Leagues
3      Players
4        Shots
5        Teams
6    TeamStats


In [10]:
# Load SQL magic
%load_ext sql

In [16]:
# Configure SQL magic display settings
%config SqlMagic.autopandas = True  # This converts results to pandas DataFrames
%config SqlMagic.feedback = False
%config SqlMagic.displaywidth = 1000  # Controls width

# Add pandas display options to control both width and length
pd.set_option('display.max_rows', 20)     # Limits number of rows
pd.set_option('display.max_columns', 20)  # Limits number of columns
pd.set_option('display.width', 1000)      # Controls overall display width
pd.set_option('display.max_colwidth', 100) # Controls maximum column width

In [12]:
# Connect to the SQLite database file
%sql sqlite:///football_database.db  


In [17]:
%%sql
SELECT COUNT(*)
FROM Leagues


 * sqlite:///football_database.db


,COUNT(*)
0,5


In [ ]:
#Using Python to automate queries across all tables as an example
for table_name in csv_files.keys():
    query = f"SELECT COUNT(*) AS row_count FROM {table_name};"
    result = pd.read_sql_query(query, conn)
    print(f"Table {table_name} has {result['row_count'][0]} rows.")

In [ ]:

%%sql
--Get all teams IDs that played in the Premier League
SELECT DISTINCT homeTeamID as id
FROM games
WHERE leagueID in (SELECT leagueID
                  FROM leagues
                  WHERE leagueID = 1)
                  



In [ ]:
%%sql
--Display every team name in the premier league
WITH premierLeagueIDs AS (SELECT DISTINCT homeTeamID as id
FROM games
WHERE leagueID in (SELECT leagueID
                  FROM leagues
                  WHERE leagueID = 1))

SELECT t.teamID, t.name
FROM teams t
WHERE t.teamID in (SELECT id
                   FROM premierLeagueIDs);



In [ ]:
%%sql
--Calculate how many goals Arsenal scored at home by season
SELECT 
    season, 
    SUM(homegoals) as Total_Home_Goals
        
FROM games g
WHERE g.homeTeamID = 83
GROUP BY season
ORDER BY season;

In [ ]:
%%sql
--Calculate how many goals Arsenal scored at home vs away from the 2014 to 2020 season
WITH Season_Performance AS (SELECT 
    season, 
    SUM(CASE WHEN homeTeamID = 83 THEN homegoals ELSE 0 END) AS Total_Home_Goals,
    SUM(CASE WHEN awayTeamID = 83 THEN awaygoals ELSE 0 END) AS Total_Away_goals
FROM games
WHERE (homeTeamID = 83
    OR awayTeamID = 83)
GROUP BY season)

SELECT *, Total_Home_Goals - Total_Away_Goals AS Goal_Difference
FROM Season_Performance
ORDER BY season;





In [ ]:
%%sql
/* Shows all winners and losers in 2020 Premier League matches with: */
SELECT  g.season, 
        g.date, 
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.homeTeamID
        ) AS Home_Team,
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.awayTeamID
        ) AS Away_Team, 
        (CASE WHEN homeGoals > awayGoals THEN "Home Team Wins"
        WHEN homeGoals < awayGoals THEN "Away Team Wins"
        ELSE 'Tie' END) As Winning_Side
FROM games g
WHERE leagueID = 1 and season = '2020'
ORDER BY date DESC;



In [ ]:
%%sql
--Same query as above writen differently
SELECT g.season, 
    g.date, 
    home.name AS HomeTeamName,
    away.name AS AwayTeamName,
    CASE 
        WHEN g.homeGoals > g.awayGoals THEN 'Home Team Wins'
        WHEN g.homeGoals < g.awayGoals THEN 'Away Team Wins'
        ELSE 'TIE' 
    END AS Winning_Side
FROM games g
INNER JOIN teams home ON g.homeTeamID = home.teamID
INNER JOIN teams away ON g.awayTeamID = away.teamID
WHERE g.leagueID = 1
ORDER BY g.date DESC;

In [ ]:
%%sql 
-- Analyzing the difference between games won at home vs games won away across leagues
WITH Winning_Sides AS (SELECT g.season, 
    g.date,
    g.leagueID,
    l.name AS League_Name,
    home.name AS HomeTeamName,
    away.name AS AwayTeamName,
    CASE 
        WHEN g.homeGoals > g.awayGoals THEN 'Home Team Wins'
        WHEN g.homeGoals < g.awayGoals THEN 'Away Team Wins'
        ELSE 'TIE' 
    END AS Winning_Side
FROM games g
INNER JOIN teams home ON g.homeTeamID = home.teamID
INNER JOIN teams away ON g.awayTeamID = away.teamID
INNER JOIN leagues l ON g.leagueID = l.leagueID
), 
Winning_Stats AS (
    SELECT 
        leagueID, 
        League_Name,
        COUNT(CASE WHEN Winning_Side = 'Home Team Wins' THEN 1 END) AS Home_Wins,
        COUNT(CASE WHEN Winning_Side = 'Away Team Wins' THEN 1 END) AS Away_Wins
    FROM Winning_Sides
    GROUP BY leagueID, League_Name
)

SELECT 
    League_Name,
    Home_wins, 
    Away_wins, 
    ROUND(CAST(Home_Wins AS FLOAT) / CAST(Away_Wins AS FLOAT),2) AS Home_Away_Ratio,
    CASE WHEN League_Name = 'TOTAL/AVG' THEN 1 ELSE 0 END AS Sort_Order
FROM Winning_Stats
UNION ALL 
SELECT 
    'TOTAL/AVG' AS League_Name, 
    SUM(Home_wins) AS Home_wins, 
    SUM(Away_wins) AS Away_wins, 
    ROUND(CAST(SUM(Home_wins) AS FLOAT) / CAST(SUM(Away_wins) AS FLOAT),2) AS Home_Away_Ratio,
    1 AS Sort_Order
FROM Winning_Stats
ORDER BY 
    Sort_Order, 
    Home_Away_Ratio DESC;

In [ ]:
%%sql
-- Calculate the sum of goals scored by each team in the Premier League 2020 and the cumulative sum of goals scored
SELECT home.name AS Home_Team, away.name AS Away_Team, g.homeGoals as Home_Goals, g.awayGoals as Away_Goals, (g.homeGoals + g.awayGoals) AS Total_Goals,
       SUM(g.homeGoals + g.awayGoals) OVER (ORDER BY g.date) AS Cumulative_Goals
FROM games g
LEFT JOIN teams home ON home.teamID = g.homeTeamID
LEFT JOIN teams away ON away.teamID = g.awayTeamID
WHERE g.leagueID = 1 AND g.season = '2020'



In [ ]:
%%sql
/*Calculate the most goals scored by a team in the Premier league 2020 indicating the team name,
the team's side, the opponent name, the date and the maximum goals*/
WITH Max_Goals_Table AS (
    SELECT MAX(homeGoals) AS Max_Goals, homeTeamID AS Team_ID, 'Home' AS Side, date, awayTeamID AS Opponent_ID
    FROM games g
    WHERE g.leagueID =1 AND g.season = '2020'

   UNION ALL

   SELECT MAX(awayGoals) AS Max_Goals, awayTeamID AS Team_ID, 'Away' AS Side, date, homeTeamID AS Opponent_ID
   FROM games g
   WHERE g.leagueID =1 AND g.season = '2020')

SELECT mg.Max_Goals AS 'Maximum Goals in a Game',
       (CASE WHEN mg.Side = 'Home' THEN home.name
       WHEN mg.Side = 'Away' THEN away.name END) AS Team_Name,
       mg.Side, 
       (CASE WHEN mg.Side = 'Home' THEN (SELECT away.name FROM teams away WHERE away.teamID = mg.Opponent_ID)
       WHEN mg.side = 'Away' THEN (SELECT home.name FROM teams home WHERE home.teamID = mg.Opponent_ID) END) AS Opponent_Name,
       strftime('%d-%m-%Y', mg.date) AS Date
FROM Max_Goals_Table mg
LEFT JOIN teams home ON home.teamID = mg.team_ID
LEFT JOIN teams away ON away.teamID = mg.team_ID
WHERE mg.Max_Goals = (SELECT MAX(Max_Goals) FROM Max_Goals_Table)


In [ ]:
%%sql
-- Calculate the moving average of goals scored per game for the past 5 games in the 2020 Premier League season
SELECT gameID AS Game_Number,(HomeGoals + AwayGoals) AS Total_Goals, AVG(HomeGoals + AwayGoals) OVER (ORDER BY date ASC ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS "5_Game_Moving_Average_Goals"
FROM games 
WHERE leagueID = 1 AND season = '2020'


In [ ]:
%%sql
-- Find the top 10 players with the highest average goals scored per game
SELECT p.name AS Player_Name, ROUND(AVG(app.goals),2) AS Average_Goals, SUM(goals) AS Total_Goals
FROM appearances app
LEFT JOIN players p ON p.playerID = app.playerID
GROUP BY p.playerID 
HAVING SUM(app.goals) IS NOT NULL and SUM(app.goals) > 10
ORDER BY Average_Goals DESC
LIMIT 10;


In [ ]:
%%sql 
-- Calculate the distribution of players across quintiles based on their average goals scored per game
WITH Quintile_Table AS (SELECT p.name AS Player_Name, ROUND(AVG(app.goals),2) AS Average_Goals, SUM(goals) AS Total_Goals, 
       NTILE(5) OVER (ORDER BY AVG(app.goals) DESC) AS Quintile
FROM appearances app
LEFT JOIN players p ON p.playerID = app.playerID
-- Group by playerID as names could potentially have duplicates and grouping by primary key is usually better
GROUP BY p.playerID 
HAVING SUM(app.goals) IS NOT NULL and SUM(app.goals) > 10
ORDER BY Average_Goals DESC)

SELECT Quintile, COUNT(*) AS Number_of_Players, ROUND(AVG(Average_Goals),2) AS Average_Goals 
FROM Quintile_Table
GROUP BY Quintile
ORDER BY Quintile;


In [ ]:
%%sql
-- Getting comprehensive statistics of player performance by 
SELECT 
    p.playerID,
    p.name as Player_Name, 
    t.teamID,
    t.name as Team_Name, 
    l.leagueID,
    l.name as League_Name, 
    g.season,
    COUNT(DISTINCT a.gameID) AS Games_Played, 
    SUM(a.goals) as Total_Goals, 
    SUM(a.assists) as Total_Assists, 
    SUM(a.shots) as Total_Shots, 
    ROUND(CAST(SUM(a.goals) AS FLOAT )/ NULLIF(SUM(a.shots),0), 2) * 100 AS Shots_Converted_Perc,
    ROUND(CAST(SUM(a.goals) AS FLOAT )/COUNT(DISTINCT a.gameID),2) AS Goals_Per_Game
FROM appearances a
LEFT JOIN players p ON p.playerID = a.playerID
LEFT JOIN games g ON g.gameID = a.gameID
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY p.playerID, p.name, t.teamID, t.name, l.leagueID, l.name, g.season
HAVING Games_Played >= 5
ORDER BY Goals_Per_Game DESC;

In [ ]:
%%sql
-- create a query to find Arsenal's ID 
SELECT *
FROM teams
WHERE name LIKE '%Arsenal%';


In [ ]:
%%sql
-- Calculate Arsenal's top scoring players by displaying goals scored, shots taken and the shot/goal ratio by season per player
SELECT p.name, p.playerID, season, SUM(a.goals) AS Goals_Scored, SUM(a.shots) AS Shots_Taken, Round(AVG(a.shots)/AVG(a.goals),3) as "Avg Shots Taken per Goal"
FROM appearances a 
LEFT JOIN games g ON a.gameID = g.gameID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
LEFT JOIN players p ON p.playerID = a.playerID 
WHERE t.teamID = 83 
GROUP BY a.playerID, g.season
ORDER BY Goals_scored DESC
LIMIT 10;


In [ ]:
%%sql
-- Rank the teams according to the number of games won over 2014 to 2020 seasons for premier league teams in the dataset 
WITH premMatches AS (SELECT  g.season, 
        g.date, 
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.homeTeamID
        ) AS Home_Team,
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.awayTeamID
        ) AS Away_Team, 
        (CASE WHEN homeGoals > awayGoals THEN "Home Team Wins"
        WHEN homeGoals < awayGoals THEN "Away Team Wins"
        ELSE 'Tie' END) As Winning_Side
FROM games g
WHERE leagueID = 1 
ORDER BY date DESC)

SELECT TEAM, SUM(Wins) AS "Total Wins", RANK () OVER (ORDER BY SUM(Wins) DESC) AS "Rank"
FROM (

SELECT Home_Team AS Team, COUNT(*) As Wins
FROM premMatches 
WHERE Winning_Side = "Home Team Wins"
GROUP BY Home_Team

UNION ALL

SELECT Away_Team AS TEAM, COUNT(*) As Wins
FROM premMatches 
WHERE Winning_Side = "Away Team Wins"
GROUP BY Away_Team
) AS Combined_Wins
GROUP BY TEAM
ORDER BY "Total Wins" DESC
LIMIT 10;


In [ ]:
%%sql 
-- Calculate the points for each team in the database
SELECT l.name AS League,
       g.season,
       t.name AS Team,
       Count(*) AS Games_Played,
       COUNT((CASE 
            --Home teams wins
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            --Away teams wins
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
       --Tie
       COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points, 
       -- Count all Wins
       COUNT(CASE 
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)  AS Wins,
       COUNT(CASE 
            WHEN homeGoals < awayGoals AND g.homeTeamID = t.teamID THEN 1 
            WHEN homeGoals > awayGoals AND g.awayTeamID = t.teamID THEN 1 END)  AS Losses, 
       COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Draws
          
FROM games g 
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
--Add the following for a specific league and season WHERE l.name = 'Premier League' AND g.season = '2020'
GROUP BY l.leagueID, g.season, t.name 
ORDER BY Points DESC


In [ ]:
%%sql 
-- Calculates detailed team performance metrics including points, goals scored/conceded, and points per game
-- for all teams across all seasons and leagues to analyze overall team performance
WITH season_points AS (SELECT 
    l.name AS League,
    g.season,
    t.name AS Team,
    COUNT((CASE 
        --Home teams wins
        WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
        --Away teams wins
        WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
    --Tie
    COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points, 
    COUNT (*) as Games_Played, 
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN homeGoals
             WHEN g.awayTeamID = t.teamID THEN awayGoals END) as Goals_For,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN awayGoals
             WHEN g.awayTeamID = t.teamID THEN homeGoals END) as Goals_Against
FROM games g 
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY l.leagueID, g.season, t.name
)

SELECT *, Goals_For - Goals_Against as Goal_Difference, 
    ROUND(CAST(points AS FLOAT)/ Games_Played, 2) As Points_Per_Game
FROM season_points

In [ ]:
%%sql 
--Display premier league winning teams showing reiging champions

WITH Season_Points AS (
    SELECT season, t.name AS Team_Name,
        SUM((CASE 
            WHEN g.hometeamID = t.teamID AND g.homeGoals > g.awayGoals THEN 3 
            WHEN g.awayteamID = t.teamID AND g.homeGoals < g.awayGoals THEN 3 END)) + 
        SUM((CASE 
            WHEN g.homeGoals = g.awayGoals THEN 1 END)) AS Season_Points
FROM games g
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
WHERE g.leagueID = 1
GROUP BY t.teamID, season
ORDER BY Season_Points DESC), 

Season_Winners AS (
    -- Identify the winner for each season 
    SELECT season, Team_Name, 
        ROW_NUMBER() OVER(PARTITION BY season ORDER BY Season_Points DESC) AS Season_Rank
    FROM Season_Points 
    GROUP BY season
)

SELECT  
    sw.season, 
    sw.Team_Name AS Champion, 
    LAG(sw.Team_Name, 1) OVER(ORDER BY season) AS Previous_Champion,
    (CASE WHEN sw.Team_Name = (LAG(sw.Team_Name, 1) OVER(ORDER BY season)) THEN 'Yes' ELSE 'No' END) AS Two_Time_Champion
FROM Season_Winners sw
WHERE sw.Season_Rank = 1 
GROUP BY sw.season
ORDER BY sw.season ASC;

In [ ]:
%%sql 
-- Analyzes league competitiveness by calculating average points gap between top/bottom teams
-- Lower points gap indicates a more competitive league

WITH season_points AS (SELECT l.name AS League,
       g.season,
       t.name AS Team,
       COUNT((CASE 
            --Home teams wins
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            --Away teams wins
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
       --Tie
       COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
FROM games g 
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY l.leagueID, g.season, t.name 
ORDER BY Points DESC)

SELECT league, 
       ROUND(AVG(Points_GAP),2) AS Avg_Points_Gap,  -- Average gap across seasons
       ROW_NUMBER() OVER (ORDER BY AVG(Points_Gap) DESC) AS Rank
FROM(
     SELECT League, Season, Max(Points) - Min(Points) AS Points_Gap -- Gap per season
     FROM season_points
     GROUP BY League, Season) Sub
GROUP BY League
ORDER BY Avg_Points_Gap DESC

In [ ]:
%%sql 
-- Analyze league competitiveness by comparing average points and points gaps between top and bottom teams across leagues
WITH season_points AS (
    SELECT 
        l.leagueID,
        l.name AS League,
        g.season,
        t.teamID,
        t.name AS Team,
        COUNT((CASE 
            --Home teams wins
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            --Away teams wins
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
        --Tie
        COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
    FROM games g 
    LEFT JOIN leagues l ON l.leagueID = g.leagueID
    LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
    GROUP BY l.leagueID, l.name, g.season, t.teamID, t.name),

league_stats AS (
    SELECT
        league,
        leagueID, 
        season,
        MAX(points) - MIN(points) AS Points_Gap_Top_to_Bottom,
        ROUND(AVG(points),2) AS Avg_Points,
        AVG(CASE WHEN Standing <= 4 THEN points END) AS Top_4_Teams_Avg_Points,
        AVG(CASE WHEN Standing >= (SELECT COUNT(*) - 3 FROM season_points sp2 
                                  WHERE sp2.league = season_standings.league 
                                  AND sp2.season = season_standings.season) 
            THEN points END) AS Bottom_4_Teams_Avg_Points
    FROM (
        SELECT 
            league,
            leagueID,
            season, 
            team,
            teamID,
            points,
            ROW_NUMBER() OVER (PARTITION BY League, season ORDER BY points DESC) as Standing
        FROM season_points
    ) AS Season_Standings
    GROUP BY league, leagueID, season
)

SELECT 
    league,
    leagueID,
    season,
    ROUND(AVG(Avg_Points),2) AS Avg_Points,
    ROUND(AVG(Points_Gap_Top_to_Bottom),2) AS Avg_Points_Gap_Top_to_Bottom,
    ROUND(AVG(Top_4_Teams_Avg_Points - Bottom_4_Teams_Avg_Points),2) AS Points_Gap_Between_Top_and_Bottom_4_Teams    
FROM league_stats
GROUP BY league, leagueID, season
ORDER BY Avg_Points DESC;

In [ ]:
# This cell analyzes the points distribution across leagues and seasons
# First, query the database to get total points for each team per season
stats_df = pd.read_sql_query("""
WITH season_points AS (
    SELECT l.name AS League,
           g.season,
           t.name AS Team,
           COUNT((CASE 
                WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
                WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
           COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
    FROM games g 
    LEFT JOIN leagues l ON l.leagueID = g.leagueID
    LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
    GROUP BY l.leagueID, g.season, t.name 
    ORDER BY Points DESC)
SELECT * FROM season_points;
""", conn)

# Calculate key statistics for each league/season:
# - Total points across all teams
# - Average points per team 
# - Standard deviation of points (measure of competitiveness)
stats = stats_df.groupby(['League', 'season'])['Points'].agg([
    ('Total_Points', 'sum'),
    ('Avg_Points', 'mean'),
    ('Std_Points', 'std')
]).round(2)

# Calculate the points gap between top and bottom teams
# This helps identify leagues with big disparities between teams
points_range = stats_df.groupby(['League', 'season'])['Points'].agg(['min', 'max'])
stats['Points_Gap'] = points_range['max'] - points_range['min']

# Sort leagues by average points to see which are highest scoring
stats = stats.sort_values('Avg_Points', ascending=False)

# Show the final analysis 
display(stats)

In [ ]:
%%sql
--Analyzing team stats per season per team for all leagues in database
WITH team_performance AS (SELECT 
    l.leagueID,
    l.name AS League, 
    ts.season, 
    t.name, 
    t.teamID,
    COUNT(*) AS Games_Played, 
    COUNT(CASE WHEN ts.result = 'W' THEN 1 END) as Wins,
    COUNT(CASE WHEN ts.result = 'D' THEN 1 END) as Draws,
    COUNT(CASE WHEN ts.result = 'L' THEN 1 END) as Losses,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN homeGoals
            WHEN g.awayTeamID = t.teamID THEN awayGoals END) as Goals_Scored,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN awayGoals
            WHEN g.awayTeamID = t.teamID THEN homeGoals END) as Goals_Conceded,
    SUM(ts.shots) AS Total_Shots,
    SUM(ts.shotsOnTarget) AS Total_Shots_On_Target, 
    SUM(ts.fouls) AS Total_Fouls, 
    SUM(ts.corners) as Total_Corners,
    SUM(ts.yellowcards) as Total_Yellow_Cards,
    SUM(ts.redcards) as Total_Red_Cards
FROM games g
LEFT JOIN leagues l on l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
LEFT JOIN teamstats ts ON ts.gameID = g.gameID -- Match the game records first 
    -- Match the teamID filed in teamstats when it is either the home team or the away team 
    AND ((ts.teamID = t.teamID and g.homeTeamID = ts.teamID)
    OR (ts.teamID = t.teamID and g.awayTeamID = ts.teamID))
GROUP BY l.name, ts.season, t.name
)

SELECT *, 
    ROUND(1.0 * Goals_Scored/ Games_Played, 2) AS Goals_Per_Game, 
    ROUND (Total_Shots_On_Target / NULLIF (Goals_Scored, 0),2) AS Shots_Per_Goal
FROM team_performance
ORDER BY season ASC, Goals_Scored DESC;


In [ ]:
#Exporting important SQL queries to csv files for data visualization in Tableau next 

# Player Performance Analysis Table
player_query = """
SELECT 
    p.playerID,
    p.name as Player_Name, 
    t.teamID,
    t.name as Team_Name, 
    l.leagueID,
    l.name as League_Name, 
    g.season,
    COUNT(DISTINCT a.gameID) AS Games_Played, 
    SUM(a.goals) as Total_Goals, 
    SUM(a.assists) as Total_Assists, 
    SUM(a.shots) as Total_Shots, 
    ROUND(CAST(SUM(a.goals) AS FLOAT )/ NULLIF(SUM(a.shots),0), 2) * 100 AS Shots_Converted_Perc,
    ROUND(CAST(SUM(a.goals) AS FLOAT )/COUNT(DISTINCT a.gameID),2) AS Goals_Per_Game
FROM appearances a
LEFT JOIN players p ON p.playerID = a.playerID
LEFT JOIN games g ON g.gameID = a.gameID
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY p.playerID, p.name, t.teamID, t.name, l.leagueID, l.name, g.season
HAVING Games_Played >= 5
ORDER BY Goals_Per_Game DESC;
"""

# Arsenal Scorer's Analysis Table
arsenal_scorers_query = """
SELECT p.name, p.playerID, season, SUM(a.goals) AS Goals_Scored, SUM(a.shots) AS Shots_Taken, Round(AVG(a.shots)/AVG(a.goals),3) as "Avg Shots Taken per Goal"
FROM appearances a 
LEFT JOIN games g ON a.gameID = g.gameID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
LEFT JOIN players p ON p.playerID = a.playerID 
WHERE t.teamID = 83 
GROUP BY a.playerID, g.season
ORDER BY Goals_scored DESC
LIMIT 10;
"""

# Team Stat Performance Analysis Table
team_stat_performance = """ 
WITH team_performance AS (SELECT 
    l.leagueID,
    l.name AS League, 
    ts.season, 
    t.name, 
    t.teamID,
    COUNT(*) AS Games_Played, 
    COUNT(CASE WHEN ts.result = 'W' THEN 1 END) as Wins,
    COUNT(CASE WHEN ts.result = 'D' THEN 1 END) as Draws,
    COUNT(CASE WHEN ts.result = 'L' THEN 1 END) as Losses,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN homeGoals
            WHEN g.awayTeamID = t.teamID THEN awayGoals END) as Goals_Scored,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN awayGoals
            WHEN g.awayTeamID = t.teamID THEN homeGoals END) as Goals_Conceded,
    SUM(ts.shots) AS Total_Shots,
    SUM(ts.shotsOnTarget) AS Total_Shots_On_Target, 
    SUM(ts.fouls) AS Total_Fouls, 
    SUM(ts.corners) as Total_Corners,
    SUM(ts.yellowcards) as Total_Yellow_Cards,
    SUM(ts.redcards) as Total_Red_Cards
FROM games g
LEFT JOIN leagues l on l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
LEFT JOIN teamstats ts ON ts.gameID = g.gameID -- Match the game records first 
    -- Match the teamID filed in teamstats when it is either the home team or the away team 
    AND ((ts.teamID = t.teamID and g.homeTeamID = ts.teamID)
    OR (ts.teamID = t.teamID and g.awayTeamID = ts.teamID))
GROUP BY l.name, ts.season, t.name
)

SELECT *, 
    ROUND(1.0 * Goals_Scored/ Games_Played, 2) AS Goals_Per_Game, 
    ROUND (Total_Shots_On_Target / NULLIF (Goals_Scored, 0),2) AS Shots_Per_Goal
FROM team_performance
ORDER BY season ASC, Goals_Scored DESC;
"""

# Export each query
for name, query in [('player_performance', player_query),
                   ('Arsenal_scorers', arsenal_scorers_query),
                   ('team_stat_performance', team_stat_performance)
                   ]:
    df = pd.read_sql_query(query, conn)
    output_path = f"C:\\Users\\Zaki Bouaoudia\\OneDrive\\09_Programming\\SQL Project\\EU Football SQL Project\\Tableau Exports\\{name}.csv"
    # Explicitly specify the mode as 'w' to overwrite
    df.to_csv(output_path, index=False, mode='w')




In [ ]:
conn = sqlite3.connect('multi_table_database.db')
# tables are already saved if using this connection
conn.close()
